In [13]:
#在计算了data_linear和data_corr，在本目录下生成了data_linear.csv和data_corr.csv后，执行该程序，进行股票模拟购买。

#整体思路为：
#1、通过线性回归中income排序数据，取正值的前1/3作为候选名单。
#2、计算候选名单相关系数大于某个值的股票的income累计，并排序，取前五个股票作为购买目标。
#3、根据购买目标的价格，购买相应数量的股票。


import time        

def main_simulate_buy():
    
    d_time = time.time() #计算程序运行耗时

    #根据相关系数和线性回归计算购买的股票，保存股票代码
    stock_code = general_stock_code()
    
    #获得购买股票的数据，并进行模拟购买
    if not stock_code == False: #如果购买股票不为空，则开始模拟购买
        
        pro = ts.pro_api('f3bbc97d0ffbbed8666e6f7c82e712165950d048987f5d6cfbf1e0ce') #token可以到“https://tushare.pro/”注册获得
        pool = pro.query('daily', ts_code = '000001.SZ') #获取最近一次交易日期，模拟将在下一个交易日期买入
        end_date = pool.trade_date[0]
        
        #如果希望获取之前的数据，直接在此处赋值截至日期。若否，将下行屏蔽掉
        #因为模型的日期统一，该日期应该与计算线性回归和相关系数的日期一致
        #end_date = '20191220' #直接赋值

        #模拟购买股票
        cal_buy(stock_code, end_date)
    
        print('完成！耗时：', time.strftime('%H:%M:%S',time.localtime(time.time()-d_time)))
    


#根据相关系数和线性回归计算购买的股票，保存股票代码
import tushare as ts
import pandas as pd
import os
    
def general_stock_code():

    print('正在计算目标股票……')

    
    #读入相关系数文件
    fn = 'data_corr.csv' #定义文件名称    

    file_exit = os.path.isfile(fn) #判断文件是否存在
    
    if file_exit == True: #如果文件存在就采用追加模式（不覆盖原文件），如果不存在就直接保存。
        corr = pd.read_csv('data_corr.csv') #读入相关系数文件
        corr_code_list = corr.columns.values.tolist() #将第一行的股票代码赋值到一个list中，以便查找及定位。corr行和列的顺序都是这个list
    else:
        print('相关系数文件：',fn,'不存在。需要先运算的到该文件，才能进行购买计算。程序终止执行！')
        return file_exit
    print('    读入相关系数……')

    
    #读入线性回归文件
    fn = 'data_linear.csv' #定义文件名称    
    
    file_exit = os.path.isfile(fn) #判断文件是否存在
    
    if file_exit == True: #如果文件存在就采用追加模式（不覆盖原文件），如果不存在就直接保存。
        linear = pd.read_csv('data_linear.csv') #读入线性回归数据文件
        linear_stock_code = list(linear.stock_code) #根据linear生成一个关于stock_code的list，供定位时调用
    else:
        print('线性回归数据文件：',fn,'不存在。需要先运算的到该文件，才能进行购买计算。程序终止执行！')
        return file_exit
    print('    读入线性回归……')

    
    #根据线性回归数据，生成一个新列表：删除income为负数的记录后，取前面1/3的数据作为候选名单。接下来再根据相关income再次排序
    print('    生成及格列表……')
    cand_list = linear[:int((len(linear))/3)] #这里认为income都不是负数，所以直接取前面1/3
    for i in range(len(linear)): #接着，判断当income为负数时，取当前数组数量的前1/3。这里，默认income已经按照从大到小排列。
        if linear.income[i] < 0:
            cand_list = linear[:int((i)/3)]
            break


    #计算相关income的累计值
    income_corr_sum = [] #定义相关income的数组
    for i in range(len(cand_list)):
        
        sum_temp = 0 #每次计算前，将相关income重置为0
        income_corr_stock_index = corr_code_list.index(cand_list.stock_code[i]) #定位当前stock_code的位置。在这里，实际上是定位到该list的行。
    
        for j in range(len(corr_code_list)-1): #在corr中查找
            if abs(corr.iloc[j, income_corr_stock_index]) > 0.9 and not corr.iloc[j, income_corr_stock_index] == 1 and corr_code_list[j] in linear_stock_code: #当相关系数大于0.9且不等于1
                sum_temp = sum_temp + linear.income[linear_stock_code.index(corr_code_list[j])] #将相关income累加
    
        income_corr_sum.append(sum_temp)            
            
            
    
    #相关income排序，然后取前5个作为购买目标
    cand_list['corr'] = income_corr_sum
    
    rank_code = pd.DataFrame(cand_list) #转为DataFrame格式

    rank_code = rank_code.sort_values('corr', ascending = False) #根据corr字段排序

    rank_code = rank_code.reset_index(drop = True) #重置索引
    
    stock_code = rank_code.stock_code.tolist()[0:5] #取前5个
    
    
    return stock_code



#模拟购买股票
import datetime
import tushare as ts
import pandas as pd
import os

def cal_buy(stock_code, end_date):
    
    pro = ts.pro_api('f3bbc97d0ffbbed8666e6f7c82e712165950d048987f5d6cfbf1e0ce') #''的token可以到“https://tushare.pro/”注册获得
    pool = pro.query('daily', ts_code = '000001.SZ') #获取最近一次交易日期，模拟将在下一个交易日期买入

    #初始化变量
    date = [end_date] * len(stock_code)
    volumn = [0] * len(stock_code)
    day_1_mid_price = [0.00] * len(stock_code)
    day_2_mid_price = [0.00] * len(stock_code)
    day_3_mid_price = [0.00] * len(stock_code)
    day_4_mid_price = [0.00] * len(stock_code)
    day_5_mid_price = [0.00] * len(stock_code)
    day_6_mid_price = [0.00] * len(stock_code)
    amount = [0.00] * len(stock_code)
    earn = [0.00] * len(stock_code)

    #计算购买数量
    for i in range(len(stock_code)):
        print('    正在计算', stock_code[i], '的购买数量……')

        try:
            pool = pro.query('daily', ts_code = stock_code[i], trade_date = date[i]) #获取股票数据
        except:
            print('连接超时，等待60秒再继续……') #普通用户，tushare每分钟只能读取200次数据。
            time.sleep(60)
            pool = pro.query('daily', ts_code = stock_code[i], trade_date = date[i]) #获取股票数据
       
        if len(pool) > 0:
        
            if round(((float(pool.high) + float(pool.low)) / 2),2) > 30 and float(pool.high) < 100: #如果股票中间价格大于30元且小于100元，则购买1手
                volumn[i] = 1
                amount[i] = float((pool.high + pool.low) / 2) * volumn[i] * 100
            elif 10 < round(((float(pool.high) + float(pool.low)) / 2),2) <= 30: #如果股票中间价格在10到30之间，则购买2手
                volumn[i] = 2
                amount[i] = float((pool.high + pool.low) / 2) * volumn[i] * 100
            elif 0 < round(((float(pool.high) + float(pool.low)) / 2),2) <= 10: #如果股票中间价格小于10元，则购买4手
                volumn[i] = 4
                amount[i] = float((pool.high + pool.low) / 2) * volumn[i] * 100
        

    #将计算结果存入字典
    print('正在保存计算结果……')
    simulate_buy = {
        'stock_code' : stock_code,
        'date' : date,
        'volumn' : volumn,
        'day_1_mid_price' :  day_1_mid_price,
        'day_2_mid_price' :  day_1_mid_price,
        'day_3_mid_price' :  day_1_mid_price,
        'day_4_mid_price' :  day_1_mid_price,
        'day_5_mid_price' :  day_1_mid_price,
        'day_6_mid_price' :  day_1_mid_price,
        'amount' : amount,
        'earn' : earn        
    }

    #将字典保存到文件
    df = pd.DataFrame(simulate_buy) #将字典转为DateFrame格式
    
    fn = 'simulate_buy.csv' #定义文件名称
    
    file_exit = os.path.isfile(fn) #判断文件是否存在
    if file_exit == True: #如果文件存在就采用追加模式（不覆盖原文件），如果不存在就直接保存。
        df_exit = pd.read_csv(fn) #首先读入已存在的文件
        if not date[0] in str(df_exit.date.tolist()): #如果计算结果中的日期不在已存在的文件中，表示可以买入，将结果追加到文件。否则跳过。
            df.to_csv(fn, mode = 'a', header = False, index = False )
            print('已追加购买记录到文件：',fn)
        else:
            print('当天已购买了股票，不能再继续购买。请查看文件：',fn)
    else:
        df.to_csv(fn, index = False)
        print('已保存购买记录到文件：',fn)
        print(df)

        
    
if __name__ == '__main__': #运行程序
    main_simulate_buy()


正在计算目标股票……
    读入相关系数……
    读入线性回归……
    生成及格列表……


d:\python\python37\lib\site-packages\ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


    正在计算 600072.SH 的购买数量……
    正在计算 600754.SH 的购买数量……
    正在计算 603596.SH 的购买数量……
    正在计算 603186.SH 的购买数量……
    正在计算 603360.SH 的购买数量……
正在保存计算结果……
已追加购买记录到文件： simulate_buy.csv
完成！耗时： 08:00:04


In [1]:
import tushare as ts
import pandas as pd
import os

def general_stock_code():

    print('正在计算目标股票……')

    #读入相关系数文件
    fn = 'data_corr.csv' #定义文件名称    

    file_exit = os.path.isfile(fn) #判断文件是否存在
    
    if file_exit == True: #如果文件存在就采用追加模式（不覆盖原文件），如果不存在就直接保存。
        corr = pd.read_csv('data_corr.csv') #读入相关系数文件
        corr_code_list = corr.columns.values.tolist() #将第一行的股票代码赋值到一个list中，以便查找及定位。corr行和列的顺序都是这个list
    else:
        print('相关系数文件：',fn,'不存在。需要先运算的到该文件，才能进行购买计算。程序终止执行！')
        return file_exit
    print('    读入相关系数……')

    #读入线性回归文件
    fn = 'data_linear.csv' #定义文件名称    
    
    file_exit = os.path.isfile(fn) #判断文件是否存在
    
    if file_exit == True: #如果文件存在就采用追加模式（不覆盖原文件），如果不存在就直接保存。
        linear = pd.read_csv('data_linear.csv') #读入线性回归数据文件
        linear_stock_code = list(linear.stock_code) #根据linear生成一个关于stock_code的list，供定位时调用
    else:
        print('线性回归数据文件：',fn,'不存在。需要先运算的到该文件，才能进行购买计算。程序终止执行！')
        return file_exit
    print('    读入线性回归……')

    #根据线性回归数据，生成一个新列表：删除income为负数的记录后，取前面1/3的数据。
    print('    生成及格列表……')
    cand_list = linear[:int((len(linear))/3)] #这里认为income都不是负数，所以直接取前面1/3
    for i in range(len(linear)): #接着，判断当income为负数时，取当前数组数量的前1/3。这里，默认income已经按照从大到小排列。
        if linear.income[i] < 0:
            cand_list = linear[:int((i)/3)]
            break
            
    return corr, corr_code_list, linear, linear_stock_code, cand_list
            
corr, corr_code_list, linear, linear_stock_code, cand_list = general_stock_code()

正在计算目标股票……
    读入相关系数……
    读入线性回归……
    生成及格列表……


In [2]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

income_corr_sum = [] #定义相关income的数组
for i in range(len(cand_list)):
   
    sum_temp = 0 #每次计算前，将相关income重置为0
    income_corr_stock_index = corr_code_list.index(cand_list.stock_code[i]) #定位当前stock_code的位置。在这里，实际上是定位到该list的行。
    
    
    for j in range(len(corr_code_list)-1): #在corr中查找
        if abs(corr.iloc[j, income_corr_stock_index]) > 0.9 and not corr.iloc[j, income_corr_stock_index] == 1 and corr_code_list[j] in linear_stock_code: #当相关系数大于0.9且不等于1
            sum_temp = sum_temp + linear.income[linear_stock_code.index(corr_code_list[j])] #将相关income累加

    print(sum_temp)
    
    income_corr_sum.append(sum_temp)

print(income_corr_sum)



-17.09633226706412
-50.485055868168416
-1.9483833589863222
-43.57779244512144
-80.53957977691014
-9.038240388582725
-11.040109415200439
2.5740240803131034
-27.801434584326984
-57.13891306539455
-49.25594813403619
0
-27.762867334828265
-74.57825825546453
0
2.4755731514740313
-46.04205351679177
-22.008467198745798
-27.602301284913473
-11.813998392291765
-53.24229787387979
-50.15656433747188
-4.442737790640724
0
0
-17.74823636378193
-5.53464647947858
-38.32138217267014
-59.53756108833438
0
0
0
-11.813998392291765
0.6344736068299417
0
-14.05994578675721
-20.537386611929
0.8528894198901469
-12.022739398348206
0
0
0.003485590487972701
-464.88422166777895
-12.022739398348206
0
-490.97367426779186
0
0
5.602821327220718
0
0
-32.17952892686655
-49.5068055276854
-14.183741764325232
0
0
0
8.248241834710832
0
-36.47583441843998
0
-119.58469191810106
-3.3427725605814977
-22.117356803245364
0
-1.9271758613281222
-912.3988035448723
-56.74933855123571
0
-18.074047959903222
-162.4584369059845
0
1.026538

In [4]:
cand_list['corr'] = income_corr_sum

d:\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
print(cand_list)

    stock_code      income  accuracy        corr
0    603160.SH  126.334662  0.989285  -17.096332
1    603501.SH   82.304792  0.990268  -50.485056
2    600745.SH   78.062847  0.988752   -1.948383
3    600519.SH   61.818036  0.996293  -43.577792
4    600763.SH   55.514783  0.994961  -80.539580
..         ...         ...       ...         ...
113  600580.SH    3.153558  0.965152    0.000000
114  600288.SH    3.149171  0.985628   -4.344624
115  603308.SH    3.137822  0.991180    2.359143
116  600323.SH    3.038475  0.966883    0.000000
117  600070.SH    2.981152  0.986171 -239.577900

[118 rows x 4 columns]


In [8]:
    #排序后取前5个
rank_code = pd.DataFrame(cand_list) #转为DataFrame格式

rank_code = rank_code.sort_values('corr', ascending = False) #根据字段排序

rank_code = rank_code.reset_index(drop = True) #重置索引
    
stock_code = rank_code.stock_code.tolist()[0:5] #取前5个

In [9]:
print(stock_code)

['600072.SH', '600754.SH', '603596.SH', '603186.SH', '603360.SH']


In [11]:
rank_code

,stock_code,income,accuracy,corr
0,600072.SH,4.488715,0.984594,50.256806
1,600754.SH,3.859011,0.974709,28.973161
2,603596.SH,3.415828,0.983113,11.456455
3,603186.SH,7.011854,0.980752,8.248242
4,603360.SH,8.054129,0.981437,5.602821
...,...,...,...,...
113,603197.SH,8.932585,0.989698,-490.973674
114,601689.SH,3.268249,0.993958,-511.434054
115,603730.SH,3.866728,0.976964,-585.542607
116,603536.SH,3.361267,0.994637,-603.817455
